## Part 2 - Deep Learning and Clustering of Image Data

In [ ]:
# Imports

In this project, we use a VGG network [6] pre-trained on the ImageNet dataset [7]. We
provide a helper codebase (check Week 4 in BruinLearn), which guides you through the
necessary steps for loading the VGG network and for using it for feature extraction.

In [ ]:
# Q19 Code

### QUESTION 19: 
In a brief paragraph discuss: If the VGG network is trained on a dataset with
perhaps totally different classes as targets, why would one expect the features derived from such a
network to have discriminative power for a custom dataset?

### ANSWER:


Use the helper code to load the flowers dataset, and extract their features. To perform
computations on deep neural networks fast enough, GPU resources are often required. GPU
resources can be freely accessed through “Google Colab”.

In [ ]:
# Q20 Code

### QUESTION 20: 
In a brief paragraph explain how the helper code base is performing feature
extraction.

### ANSWER:


### QUESTION 21:
How many pixels are there in the original images? How many features does
the VGG network extract per image; i.e what is the dimension of each feature vector for an image
sample?

### ANSWER:


### QUESTION 22:
Are the extracted features dense or sparse? (Compare with sparse TF-IDF
features in text.)

### ANSWER:


### QUESTION 23:
In order to inspect the high-dimensional features, t-SNE is a popular off-the-shelf
choice for visualizing Vision features. Map the features you have extracted onto 2 dimensions with
t-SNE. Then plot the mapped feature vectors along x and y axes. Color-code the data points with
ground-truth labels. Describe your observation.

In [ ]:
# Q23 Code

While PCA is a powerful method for dimensionality reduction, it is limited to “linear”
transformations. This might not be particularly good if a dataset is distributed non-linearly.
An alternative approach is use of an “autoencoder” or UMAP. The helper has implemented an
autoencoder which is ready to use.

In [ ]:
# Code here

### QUESTION 24:
Report the best result (in terms of rand score) within the table below.
For HDBSCAN, introduce a conservative parameter grid over min cluster size and min samples.

In [ ]:
# Code for Q24

# ANSWER:

| Module | Alternatives | Hyperparameters |
| :---: | ----- | ----- |
|  Dimensionality Reduction | None | N/A |
|  | SVD | r \= 50 |
|  | UMAP | n\_components \= 50 |
|  | Autoencoder | num\_features \= 50 |
|  Clustering | K-Means | k \= 5 |
|  | Agglomerative Clustering | n\_clusters \= 5 |
|  | HDBSCAN | Min\_cluster\_size & min\_samples |

Lastly, we can conduct an experiment to ensure that VGG features are rich enough in
information about the data classes. In particular, we can train a fully-connected neural network
classifier to predict the labels of data. For this task, you may use the MLP4 module provided in
the helper code base.

In [ ]:
# Code here

### QUESTION 25:
Report the test accuracy of the MLP classifier on the original VGG features.
Report the same when using the reduced-dimension features (you have freedom in choosing the
dimensionality reduction algorithm and its parameters). Does the performance of the model suffer
with the reduced-dimension representations? Is it significant? Does the success in classification
make sense in the context of the clustering results obtained for the same features in Question 24

In [ ]:
# Code for Q24

### ANSWER:
